## Домашнее задание 8

#### **Задание**
Данные берем отзывы за лето.

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурирующими выяснить какая архитектура больше подходит для нашей задачи 
1. построить свёрточные архитектуры 
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

## Решение:

Установим и импортируем необходимые библиотеки.

In [ ]:
!pip install natasha stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 193 kB/s 
     |████████████████████████████████| 41 kB 147 kB/s 
     |████████████████████████████████| 55 kB 4.6 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 8.2 MB 22.0 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=3aacbb02924c3b355bb3a914eb0a7993c4c80b449fad6e02a0c74db17d9022fb
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=57a603bd72cb8434d039e8aa769cba67cb11b52ba41f4282d9720ffda6174722
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import keras

import warnings
warnings.filterwarnings("ignore")

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping

Загрузим датасеты с текстами и проведем предобработку, аналогичную показанной на лекции.

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("val.csv")

df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [ ]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [ ]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

y_train = df_train['class'].values
y_val = df_val['class'].values

Сначала построим сверточную сеть.

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length, trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(256, 3, padding='same'))
model.add(Conv1D(128, 3, padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling1D())
model.add(Conv1D(64, 3, padding='same'))
# model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 30)            7743240   
                                                                 
 masking (Masking)           (None, 27, 30)            0         
                                                                 
 conv1d (Conv1D)             (None, 27, 256)           23296     
                                                                 
 conv1d_1 (Conv1D)           (None, 27, 128)           98432     
                                                                 
 activation (Activation)     (None, 27, 128)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 13, 128)          0         
 )                                                               
                                                        

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_split=0.15)

Epoch 1/10
4821/4821 [==============================] - 56s 9ms/step - loss: 0.6772 - accuracy: 0.5798 - val_loss: 0.6432 - val_accuracy: 0.6245
Epoch 2/10
4821/4821 [==============================] - 43s 9ms/step - loss: 0.6024 - accuracy: 0.6639 - val_loss: 0.5876 - val_accuracy: 0.6762
Epoch 3/10
4821/4821 [==============================] - 43s 9ms/step - loss: 0.5485 - accuracy: 0.7121 - val_loss: 0.5609 - val_accuracy: 0.7025
Epoch 4/10
4821/4821 [==============================] - 46s 9ms/step - loss: 0.5082 - accuracy: 0.7445 - val_loss: 0.5442 - val_accuracy: 0.7176
Epoch 5/10
4821/4821 [==============================] - 43s 9ms/step - loss: 0.4750 - accuracy: 0.7695 - val_loss: 0.5361 - val_accuracy: 0.7247
Epoch 6/10
4821/4821 [==============================] - 43s 9ms/step - loss: 0.4462 - accuracy: 0.7889 - val_loss: 0.5325 - val_accuracy: 0.7306
Epoch 7/10
4821/4821 [==============================] - 43s 9ms/step - loss: 0.4198 - accuracy: 0.8055 - val_loss: 0.5347 - val_ac

In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=32, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

709/709 [==============================] - 2s 3ms/step - loss: 0.5703 - accuracy: 0.7332


Test score: 0.570338249206543
Test accuracy: 0.7331922650337219


Теперь построим несколько сетей с рекурентными блоками.

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30, trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(96, recurrent_dropout=0.2))
model.add(Dense(40, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy', 
    metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=5,
                    verbose=1,
                    validation_split=0.15)

Epoch 1/5
603/603 [==============================] - 38s 60ms/step - loss: 0.7034 - accuracy: 0.5056 - val_loss: 0.6890 - val_accuracy: 0.5398
Epoch 2/5
603/603 [==============================] - 36s 60ms/step - loss: 0.6812 - accuracy: 0.5577 - val_loss: 0.6566 - val_accuracy: 0.6369
Epoch 3/5
603/603 [==============================] - 35s 59ms/step - loss: 0.5850 - accuracy: 0.6992 - val_loss: 0.5376 - val_accuracy: 0.7250
Epoch 4/5
603/603 [==============================] - 36s 59ms/step - loss: 0.4540 - accuracy: 0.7984 - val_loss: 0.5139 - val_accuracy: 0.7422
Epoch 5/5
603/603 [==============================] - 36s 59ms/step - loss: 0.3524 - accuracy: 0.8592 - val_loss: 0.5303 - val_accuracy: 0.7432


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=256, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

89/89 [==============================] - 1s 6ms/step - loss: 0.5283 - accuracy: 0.7393


Test score: 0.5283211469650269
Test accuracy: 0.739276111125946


Модель с LSTM блоками.

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30, trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(128, recurrent_dropout=0.2))
# model.add(LSTM(64, recurrent_dropout=0.4))
model.add(Dense(40, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=7e-5),
    loss='binary_crossentropy', 
    metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_2 (Masking)         (None, 27, 30)            0         
                                                                 
 lstm (LSTM)                 (None, 128)               81408     
                                                                 
 dense_4 (Dense)             (None, 40)                5160      
                                                                 
 dropout_1 (Dropout)         (None, 40)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 41        
                                                                 
Total params: 7,829,849
Trainable params: 7,829,849
No

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.15)

Epoch 1/5
302/302 [==============================] - 44s 139ms/step - loss: 0.6845 - accuracy: 0.5711 - val_loss: 0.6639 - val_accuracy: 0.6055
Epoch 2/5
302/302 [==============================] - 42s 138ms/step - loss: 0.6154 - accuracy: 0.6615 - val_loss: 0.5559 - val_accuracy: 0.7139
Epoch 3/5
302/302 [==============================] - 41s 136ms/step - loss: 0.5114 - accuracy: 0.7545 - val_loss: 0.5219 - val_accuracy: 0.7387
Epoch 4/5
302/302 [==============================] - 42s 139ms/step - loss: 0.4405 - accuracy: 0.8053 - val_loss: 0.5148 - val_accuracy: 0.7447
Epoch 5/5
302/302 [==============================] - 43s 143ms/step - loss: 0.3798 - accuracy: 0.8434 - val_loss: 0.5157 - val_accuracy: 0.7476


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 13ms/step - loss: 0.5335 - accuracy: 0.7361


Test score: 0.5335353016853333
Test accuracy: 0.7360578179359436


Результаты для этих моделей оказались чуть-чуть лучше, чем для сверточного варианта. Теперь построим сеть, в которой будут скомбинированы сверточные и рекурентные блоки.

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30, trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(128, recurrent_dropout=0.3, return_sequences=True))
model.add(Conv1D(256, 3, padding='same'))
# model.add(Conv1D(128, 5, padding='same'))
# model.add(Activation("relu"))
model.add(MaxPooling1D())
# model.add(Conv1D(128, 3, padding='same'))
# model.add(Activation("relu"))
# model.add(LSTM(96, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(32, recurrent_dropout=0.5))
# model.add(LSTM(64, recurrent_dropout=0.4))
model.add(Dense(40, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy', 
    metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_3 (Masking)         (None, 27, 30)            0         
                                                                 
 lstm_1 (LSTM)               (None, 27, 128)           81408     
                                                                 
 conv1d_3 (Conv1D)           (None, 27, 256)           98560     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 256)          0         
 1D)                                                             
                                                                 
 lstm_2 (LSTM)               (None, 32)                36992     
                                                      

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=3,
                    verbose=1,
                    validation_split=0.15)

Epoch 1/3
2411/2411 [==============================] - 416s 171ms/step - loss: 0.5925 - accuracy: 0.6703 - val_loss: 0.5217 - val_accuracy: 0.7346
Epoch 2/3
2411/2411 [==============================] - 407s 169ms/step - loss: 0.4369 - accuracy: 0.8035 - val_loss: 0.5113 - val_accuracy: 0.7473
Epoch 3/3
2411/2411 [==============================] - 407s 169ms/step - loss: 0.3090 - accuracy: 0.8767 - val_loss: 0.5628 - val_accuracy: 0.7423


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=64, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

355/355 [==============================] - 7s 18ms/step - loss: 0.5931 - accuracy: 0.7335


Test score: 0.5931180119514465
Test accuracy: 0.7334567904472351


Как видим добиться какого-то качественного улучшения модели благодаря комбинации сверточных и рекурентных слоев не удалось. Возможно более продолжительный подбор гиперпараметров позволит несколько улучшить результаты на тестовой выборке, тем не менее какого-то сильного прироста данная модель не дает.